Based on Estimating local and global feature importance scores using DiCE:
http://interpret.ml/DiCE/notebooks/DiCE_feature_importances.html#Local-feature-importance

In [5]:
import sys
sys.path.append("/mnt/nvme2tb/ffp/code/mlfires/ML_fires_al/")
import pathlib

import pandas as pd
import dice_ml
from dice_ml import Dice

from best_models import retrieve_models_by_id
from check_and_prepare_dataset import load_dataset
from manage_model import mm_load_model

2023-10-03 10:53:45.489894: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
data_dir = "/mnt/nvme2tb/ffp/datasets"
models_dir = "/mnt/nvme2tb/ffp/results/bestmodels"

filters = ["df_flt['params'].str.contains(\"'dropout': None\")"]
opt_targets = [
    "auc", "f1-score 1", "hybrid1", "hybrid2", "hybrid5", "NH2", "NH5", "NH10"
]
testfpattern = cvrespattern = "*NN_ns*mean*"
testmodel = retrieve_models_by_id(785, "hybrid2 test", models_dir, testfpattern,
                                  opt_targets, "val.", "test", filters, 3)

x_train, y_train, _ = load_dataset(pathlib.Path(data_dir,
                                                "train/train_new_sample_1_2_norm.csv"),
                             testmodel["params"]["feature_drop"])
x_train = x_train.reindex(sorted(x_train.columns), axis=1)

Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_corine_gr33', 'bin_corine_gr23', 'bin_weekday_2', 'bin_dir_max_3', 'bin_month_10', 'bin_month_9', 'bin_weekday_6', 'bin_weekday_5', 'bin_corine_gr24', 'bin_dom_dir_7', 'bin_weekday_4', 'pop', 'bin_month_3', 'bin_corine_gr21', 'bin_dir_max_8', 'bin_weekday_3', 'bin_dom_dir_8', 'bin_month_5', 'bin_dom_dir_2', 'bin_corine_gr31', 'bin_corine_gr1', 'bin_corine_gr22', 'bin_dir_max_1', 'bin_corine_gr4', 'bin_month_6', 'bin_dom_dir_3', 'bin_month_7', 'bin_dir_max_2', 'bin_corine_gr5', 'bin_corine_gr32', 'bin_weekday_7', 'bin_dir_max_7', 'bin_dom_dir_4', 'bin_dir_max_6', 'bin_month_4', 'bin_dom_dir_6', 'bin_weekday_1', 'bin_dom_dir_5', 'bin_month_8', 'bin_dom_dir_1', 'bin_dir_ma

In [8]:
x_test, y_test, _ = load_dataset(pathlib.Path(
    data_dir, "test/2019/20190901_df_norm.csv"),
                                 testmodel["params"]["feature_drop"])
x_test = x_test.reindex(sorted(x_test.columns), axis=1)

Loading full dataset /mnt/nvme2tb/ffp/datasets/test/2019/20190901_df_norm.csv
before nan drop: 656089
after nan drop: 656089
after dup. drop: 656089
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'id', 'firedate']
Dropped columns ['bin_corine_gr33', 'bin_corine_gr23', 'bin_weekday_2', 'bin_dir_max_3', 'bin_month_10', 'bin_month_9', 'bin_weekday_6', 'bin_weekday_5', 'bin_corine_gr24', 'bin_dom_dir_7', 'bin_weekday_4', 'pop', 'bin_month_3', 'bin_corine_gr21', 'bin_dir_max_8', 'bin_weekday_3', 'bin_dom_dir_8', 'bin_month_5', 'bin_dom_dir_2', 'bin_corine_gr31', 'bin_corine_gr1', 'bin_corine_gr22', 'bin_dir_max_1', 'bin_corine_gr4', 'bin_month_6', 'bin_dom_dir_3', 'bin_month_7', 'bin_dir_max_2', 'bin_corine_gr5', 'bin_corine_gr32', 'bin_weekday_7', 'bin_dir_max_7', 'bin_dom_dir_4', 'bin_dir_max_6', 'bin_month_4', 'bin_dom_dir_6', 'bin_weekday_1', 'bin_dom_dir_5', 'bin_month_8', 'bin_dom_dir_1', 'bin_di

In [9]:
dataset = pd.concat([pd.concat([x_train, y_train], axis=1),
                     pd.concat([x_test, y_test], axis=1)])
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset

,aspect,bin_corine_1,bin_corine_10,bin_corine_11,bin_corine_12,bin_corine_13,bin_corine_14,bin_corine_15,bin_corine_16,bin_corine_17,...,min_dew_temp,min_temp,ndvi,rain_7_days,res_max,road_dens,slope,xpos,ypos,fire
0,0.164706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.764075,0.815988,0.830583,0.001377,0.057340,0.188665,0.988950,0.397291,0.861254,0.0
1,0.423529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.646270,0.678533,0.855500,0.000241,0.075774,0.018803,0.961326,0.513552,0.893197,0.0
2,0.164706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.747818,0.798063,0.771333,0.000030,0.220898,0.061562,0.994475,0.339410,0.711704,0.0
3,0.184314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.577074,0.762813,0.511417,0.000312,0.072793,0.000000,1.000000,0.198699,0.914976,0.0
4,0.090196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.664033,0.773048,0.713250,0.000000,0.278515,0.000000,1.000000,0.397790,0.375578,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697768,0.250980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.722482,0.798307,0.411667,0.000030,0.186953,0.088072,0.795580,0.162773,0.560701,0.0
697769,0.607843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.610252,0.717933,0.628917,0.001049,0.135484,0.022208,0.994475,0.256082,0.727675,0.0
697770,0.941176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.761490,0.775785,0.693417,0.000000,0.237672,0.033159,1.000000,0.457168,0.064862,0.0
697771,0.435294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.602220,0.687547,0.620583,0.001204,0.104828,0.031294,0.961326,0.284024,0.560701,0.0


In [10]:
model_file = "hypres_tf_ns_ncv_do_2019_model_id_785_r_0_hybrid2test_1.h5"
weights_file = "hypres_tf_ns_ncv_do_2019_weights_id_785_r_0_hybrid2test_1.cpkt"

model = mm_load_model(pathlib.Path(models_dir, "entiremodels", model_file),
                      "tf", testmodel["params"], x_train)
model.load_weights(pathlib.Path(models_dir, "weights", weights_file))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1300)              83200     
                                                                 
 dense_1 (Dense)             (None, 2)                 2602      
                                                                 
Total params: 85,802
Trainable params: 85,802
Non-trainable params: 0
_________________________________________________________________


2023-10-03 10:56:50.413619: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
d = dice_ml.Data(dataframe=dataset,
                 continuous_features=list(dataset.columns)[:-1],
                 outcome_name="fire")
m = dice_ml.Model(model=model, backend="TF2")

Local feature importance - generate counterfactuals for a given input point.

In [12]:
exp = Dice(d, m, method="random")
query_instance = x_test[1:2]
e1 = exp.generate_counterfactuals(query_instance, total_CFs=10, desired_range=None,
                                  desired_class="opposite",
                                  permitted_range=None, features_to_vary="all")
e1.visualize_as_dataframe(show_only_changes=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.49s/it]

Query instance (original outcome : 0)


,aspect,bin_corine_1,bin_corine_10,bin_corine_11,bin_corine_12,bin_corine_13,bin_corine_14,bin_corine_15,bin_corine_16,bin_corine_17,...,min_dew_temp,min_temp,ndvi,rain_7_days,res_max,road_dens,slope,xpos,ypos,fire
0,0.290196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.530822,0.662114,0.646583,0.004122,0.080139,0.0,0.98895,0.338412,0.983944,0



Diverse Counterfactual set (new outcome: 1.0)


,aspect,bin_corine_1,bin_corine_10,bin_corine_11,bin_corine_12,bin_corine_13,bin_corine_14,bin_corine_15,bin_corine_16,bin_corine_17,...,min_dew_temp,min_temp,ndvi,rain_7_days,res_max,road_dens,slope,xpos,ypos,fire
0,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
1,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
2,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
3,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
4,0.2901960784313726,-,-,-,-,-,-,-,1.0,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
5,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.74687048,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
6,0.2901960784313726,-,-,-,-,-,0.6,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.72587523,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
7,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.66926054,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
8,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
9,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0


These can now be used to calculate the feature importance scores.

In [13]:
imp = exp.local_feature_importance(query_instance,
                                   cf_examples_list=e1.cf_examples_list)
print(imp.local_importance)

[{'bin_corine_29': 0.3, 'bin_corine_32': 0.3, 'res_max': 0.3, 'bin_corine_21': 0.2, 'bin_corine_14': 0.1, 'bin_corine_16': 0.1, 'bin_corine_2': 0.1, 'bin_corine_25': 0.1, 'bin_corine_26': 0.1, 'bin_corine_33': 0.1, 'bin_corine_36': 0.1, 'bin_corine_7': 0.1, 'aspect': 0.0, 'bin_corine_1': 0.0, 'bin_corine_10': 0.0, 'bin_corine_11': 0.0, 'bin_corine_12': 0.0, 'bin_corine_13': 0.0, 'bin_corine_15': 0.0, 'bin_corine_17': 0.0, 'bin_corine_18': 0.0, 'bin_corine_19': 0.0, 'bin_corine_20': 0.0, 'bin_corine_22': 0.0, 'bin_corine_23': 0.0, 'bin_corine_24': 0.0, 'bin_corine_27': 0.0, 'bin_corine_28': 0.0, 'bin_corine_3': 0.0, 'bin_corine_30': 0.0, 'bin_corine_31': 0.0, 'bin_corine_35': 0.0, 'bin_corine_37': 0.0, 'bin_corine_38': 0.0, 'bin_corine_4': 0.0, 'bin_corine_40': 0.0, 'bin_corine_41': 0.0, 'bin_corine_42': 0.0, 'bin_corine_43': 0.0, 'bin_corine_44': 0.0, 'bin_corine_5': 0.0, 'bin_corine_6': 0.0, 'bin_corine_8': 0.0, 'bin_corine_9': 0.0, 'dem': 0.0, 'dom_vel': 0.0, 'evi': 0.0, 'f81': 0.0, 

Feature importance can also be estimated directly, by leaving the cf_examples_list argument blank.

In [14]:
imp = exp.local_feature_importance(query_instance, posthoc_sparsity_param=None)
print(imp.local_importance)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.13it/s]

[{'bin_corine_44': 0.3, 'bin_corine_14': 0.2, 'bin_corine_16': 0.2, 'bin_corine_20': 0.2, 'bin_corine_26': 0.2, 'bin_corine_33': 0.2, 'bin_corine_24': 0.1, 'bin_corine_37': 0.1, 'lst_day': 0.1, 'lst_night': 0.1, 'res_max': 0.1, 'aspect': 0.0, 'bin_corine_1': 0.0, 'bin_corine_10': 0.0, 'bin_corine_11': 0.0, 'bin_corine_12': 0.0, 'bin_corine_13': 0.0, 'bin_corine_15': 0.0, 'bin_corine_17': 0.0, 'bin_corine_18': 0.0, 'bin_corine_19': 0.0, 'bin_corine_2': 0.0, 'bin_corine_21': 0.0, 'bin_corine_22': 0.0, 'bin_corine_23': 0.0, 'bin_corine_25': 0.0, 'bin_corine_27': 0.0, 'bin_corine_28': 0.0, 'bin_corine_29': 0.0, 'bin_corine_3': 0.0, 'bin_corine_30': 0.0, 'bin_corine_31': 0.0, 'bin_corine_32': 0.0, 'bin_corine_35': 0.0, 'bin_corine_36': 0.0, 'bin_corine_38': 0.0, 'bin_corine_4': 0.0, 'bin_corine_40': 0.0, 'bin_corine_41': 0.0, 'bin_corine_42': 0.0, 'bin_corine_43': 0.0, 'bin_corine_5': 0.0, 'bin_corine_6': 0.0, 'bin_corine_7': 0.0, 'bin_corine_8': 0.0, 'bin_corine_9': 0.0, 'dem': 0.0, 'dom_v

Global importance - For global importance, we need to generate counterfactuals for a representative sample of the dataset.

In [15]:
x_test_ = x_test.sample(1000).reset_index(drop=True)

cobj = exp.global_feature_importance(x_test_, total_CFs=10,
                                     posthoc_sparsity_param=None)
print(cobj.summary_importance)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:49<00:00,  2.13it/s]


{'bin_corine_33': 0.1085, 'bin_corine_27': 0.0734, 'res_max': 0.0627, 'bin_corine_26': 0.0566, 'bin_corine_32': 0.0538, 'bin_corine_42': 0.0519, 'bin_corine_28': 0.0447, 'rain_7_days': 0.0394, 'bin_corine_20': 0.0381, 'bin_corine_29': 0.0377, 'bin_corine_6': 0.0365, 'bin_corine_25': 0.036, 'bin_corine_10': 0.0349, 'bin_corine_31': 0.0334, 'f81': 0.0334, 'bin_corine_35': 0.0332, 'bin_corine_14': 0.0325, 'bin_corine_43': 0.0312, 'bin_corine_44': 0.0307, 'bin_corine_24': 0.0298, 'bin_corine_18': 0.0296, 'dem': 0.0291, 'bin_corine_15': 0.0279, 'bin_corine_8': 0.0276, 'bin_corine_37': 0.0266, 'bin_corine_21': 0.0265, 'bin_corine_38': 0.0257, 'bin_corine_17': 0.0256, 'bin_corine_7': 0.0254, 'frequency': 0.0251, 'evi': 0.025, 'bin_corine_1': 0.0244, 'bin_corine_23': 0.0227, 'bin_corine_12': 0.0218, 'min_dew_temp': 0.0218, 'bin_corine_40': 0.0217, 'bin_corine_2': 0.0212, 'bin_corine_4': 0.0202, 'bin_corine_9': 0.0199, 'max_temp': 0.0192, 'dom_vel': 0.0182, 'bin_corine_16': 0.0181, 'xpos': 0.01